<a href="https://colab.research.google.com/github/isabelladosanjos/SymphonyIA_ProjetoADS/blob/main/SymphonyIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.35.6 crewai-tools==0.1.7 langchain-google-genai==1.0.4 google-generativeai==0.5.4 google-search-results==2.4.2

print("\n--- Bibliotecas com versões compatíveis instaladas com sucesso! ---")

In [ ]:
import os
from google.colab import userdata
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from langchain_google_genai import ChatGoogleGenerativeAI

# Configurando as chaves de API
try:
    google_api_key = userdata.get("GOOGLE_API_KEY")
    serper_api_key = userdata.get("SERPER_API_KEY")
    os.environ["GOOGLE_API_KEY"] = google_api_key
    os.environ["SERPER_API_KEY"] = serper_api_key
    print("--- Chaves de API configuradas com sucesso! ---")
except Exception as e:
    print(f"Erro ao configurar as chaves de API: {e}")

# Ferramenta de busca
search_tool = SerperDevTool()

# Inicialização explícita do LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest", # Revertendo para o modelo mais atualizado, agora que o problema do prefixo foi corrigido.
    verbose=True,
    temperature=0.6,
    google_api_key=google_api_key
)

# Definindo o Pedido do Usuário
pedido_usuario = "Estou me sentindo meio pra baixo hoje, com saudade de casa. Queria algo acústico, talvez um folk ou um indie calmo, que me abrace."
print(f"\n--- Pedido do Usuário: '{pedido_usuario}' ---")

# Criando os Agentes
sentiment_analyzer_agent = Agent(
    role='Analisador de Sentimentos Musicais',
    goal=f"Analisar o texto do usuário para extrair o sentimento principal, o contexto e as características musicais desejadas.",
    backstory="Você é um psicólogo com uma paixão por música, especialista em traduzir emoções em atributos musicais.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

music_expert_agent = Agent(
    role='Especialista em Descobertas Musicais',
    goal="Encontrar 3 artistas ou músicas pouco conhecidos que se encaixem perfeitamente no pedido, com uma breve explicação.",
    backstory="Você é um curador de música lendário que foge do óbvio para criar experiências memoráveis, focado em encontrar jóias escondidas.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=llm
)

# Criando as Tarefas

analysis_task = Task(
    description=f"Analise o seguinte pedido de um usuário: '{pedido_usuario}'. Identifique o sentimento, contexto e atributos musicais chave (gênero, ritmo, humor, instrumentação).",
    expected_output="Um parágrafo curto e claro resumindo a análise, indicando sentimento, contexto e os atributos musicais desejados.",
    agent=sentiment_analyzer_agent
)

# ===== MUDANÇAS IMPORTANTES AQUI NA TAREFA DE RECOMENDAÇÃO =====
recommendation_task = Task(
    description="""Usando a análise de sentimento da tarefa anterior, pesquise na internet por 3 músicas ou artistas *pouco conhecidos* que se encaixem perfeitamente.
    Para cada recomendação:
    1. Forneça o **Nome do Artista** e o **Nome da Música** (se possível, ou um álbum representativo).
    2. Escreva um **parágrafo curto e convincente** explicando *por que* essa escolha é ideal para o sentimento e o pedido do usuário (tristeza, saudade, acústico, folk/indie calmo, que abrace).
    3. **Priorize artistas e músicas que não sejam mainstream.** Se os resultados iniciais forem muito famosos, faça uma nova busca mais específica.
    """,
    expected_output="""Uma lista formatada com 3 recomendações de artistas/músicas, cada uma com:
    - **Artista/Música:** [Nome do Artista] - "[Nome da Música]"
    - **Por que combina com você:** [Explicação convincente de 2-3 frases].

    Exemplo de formato para o output:
    'Aqui estão 3 recomendações que parecem perfeitas para o seu momento:

    1.  **Artista/Música:** Damien Rice - "Cannonball"
        * **Por que combina com você:** A melodia suave e a voz emotiva de Damien Rice capturam a essência da melancolia e da saudade. A instrumentação acústica e a profundidade lírica oferecem um abraço sonoro, ideal para momentos de introspecção.

    2.  **Artista/Música:** Iron & Wine - "Flightless Bird, American Mouth"
        * **Por que combina com você:** Com sua sonoridade folk acústica e vocais sussurrantes, esta música cria uma atmosfera íntima e reconfortante. A letra poética e a progressão melódica trazem uma sensação de calma e aceitação para o sentimento de saudade.

    3.  **Artista/Música:** José González - "Heartbeats"
        * **Por que combina com você:** A guitarra clássica e a voz tranquila de José González proporcionam uma experiência minimalista e profundamente emotiva. A canção induz a um estado de paz reflexiva, perfeita para encontrar consolo na tristeza e na nostalgia de casa.'
    """,
    agent=music_expert_agent
)

# Montando a Equipe e Executando
music_discovery_crew = Crew(
    agents=[sentiment_analyzer_agent, music_expert_agent],
    tasks=[analysis_task, recommendation_task],
    process=Process.sequential,
    verbose=True
)

print("\n## Iniciando a sua descoberta musical...")
result = music_discovery_crew.kickoff()

# Exibindo o Resultado Final
print("\n\n--------------------------------------------------")
print("## Sua Trilha Sonora Personalizada Está Pronta:")
print("--------------------------------------------------")
print(result)

--- Chaves de API configuradas com sucesso! ---

--- Pedido do Usuário: 'Estou me sentindo meio pra baixo hoje, com saudade de casa. Queria algo acústico, talvez um folk ou um indie calmo, que me abrace.' ---

## Iniciando a sua descoberta musical...
 [2025-09-24 23:04:03][DEBUG]: == Working Agent: Analisador de Sentimentos Musicais
 [2025-09-24 23:04:03][INFO]: == Starting Task: Analise o seguinte pedido de um usuário: 'Estou me sentindo meio pra baixo hoje, com saudade de casa. Queria algo acústico, talvez um folk ou um indie calmo, que me abrace.'. Identifique o sentimento, contexto e atributos musicais chave (gênero, ritmo, humor, instrumentação).


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: O usuário expressa um sentimento de tristeza e melancolia ("meio pra baixo hoje, com saudade de casa"),  situado no contexto de solidão e nostalgia da casa.  Musicalmente, busca-se conforto e familiaridade,  desejando um som acústico, com gên